# LDA Topic Modelling

In [ ]:
#natural language toolkit for stopwords

import nltk
nltk.download("stopwords")

import numpy as np
import json
import glob

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess

# validation
from gensim.models import CoherenceModel

#spacy
import spacy
# you may have to run in terminal: python3 -m spacy download en
from nltk.corpus import stopwords

#visualizing the data
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

#surpress some useless warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from tqdm import tqdm

[nltk_data] Downloading package stopwords to /home/ckoss/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Load Data

In [ ]:
import sqlite3, pandas as pd, src
 
db = sqlite3.connect(src.PATH / "data/example.db")
cur = db.cursor()
df = pd.DataFrame(cur.execute("""select * from abstracts""").fetchall(), columns=[c[0] for c in cur.description])
df.head()

,pk_abstracts,fk_items,text
0,24957473,1451557,Both psychosis and anhedonia have been associated to some extent with striatal functioning. The current study examined wheth...
1,24958118,5420251,This article introduces a new approach for evaluating replication results. It combines effect-size estimation with hypothesi...
2,24958343,6761399,"Major depressive disorder (MDD) is a devastating disease affecting over 300 million people worldwide, and costing an estimat..."
3,24959375,13397708,Posttraumatic stress disorder (PTSD) is associated with regional alterations in brain structure and function that are hypoth...
4,24961506,26129139,This article offers a new model for bulimia nervosa (BN) that explains both the initial impulsive nature of binge eating and...


In [ ]:
stopwords = stopwords.words("english")
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# Lemmatize and Clean text

In [ ]:
#reduce words to nouns, adjectives, verbs, adverbes and transform to root 

def lemmatization(texts, allowed_postags=["NOUN", "ADJ"]):#,"VERB","ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser","ner"]) #reduce expensive computation
    texts_out = []
    
    for text in tqdm(texts):
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_) #lemma = lemmatized form
                
        final = " ".join(new_text)
        texts_out.append(final)
    return texts_out       

lemmatized_texts = lemmatization(df['text'])


100%|██████████| 5557/5557 [01:37<00:00, 56.79it/s]


In [ ]:
print(df['text'][0])

Both psychosis and anhedonia have been associated to some extent with striatal functioning. The current study examined whether either psychosis risk or social anhedonia was associated with performance on 3 tasks related to striatal functioning. Psychosis risk participants had extremely elevated Perceptual Aberration/Magical Ideation (PerMag) scores (n = 69), with 43% of psychosis risk participants also having semistructured interview-assessed psychotic-like experiences which further heightens their risk of psychotic disorder (Chapman, Chapman, Kwapil, Eckblad, & Zinser, 1994). Compared with both extremely elevated social anhedonia (n = 60) and control (n = 68) groups, the PerMag group exhibited poorer performance on 2 of the striatum-related tasks, the Weather Prediction Task (WPT) and the Learned Irrelevance Paradigm, but not on Finger Tapping. In addition, PerMag participants with psychotic-like experiences were especially impaired on the WPT. Overall, this study arguably provides th

In [ ]:
lemmatized_texts[0]

'psychosis extent striatal functioning current study psychosis risk social performance task striatal functioning psychosis risk participant elevated score % psychosis risk participant interview psychotic like experience risk psychotic disorder elevated social anhedonia control n group group poor performance task addition permag participant psychotic like experience study first evidence psychosis risk social performance task activation associative striatum useful behavioral measure psychosis risk'

In [ ]:
def gen_words(texts):
    final = []
    for text in tqdm(texts):
        new = gensim.utils.simple_preprocess(text, deacc=True) #deacc = remove accents
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

100%|██████████| 5557/5557 [00:02<00:00, 2603.23it/s]


In [ ]:
print(data_words[0])

['psychosis', 'extent', 'striatal', 'functioning', 'current', 'study', 'psychosis', 'risk', 'social', 'performance', 'task', 'striatal', 'functioning', 'psychosis', 'risk', 'participant', 'elevated', 'score', 'psychosis', 'risk', 'participant', 'interview', 'psychotic', 'like', 'experience', 'risk', 'psychotic', 'disorder', 'elevated', 'social', 'anhedonia', 'control', 'group', 'group', 'poor', 'performance', 'task', 'addition', 'permag', 'participant', 'psychotic', 'like', 'experience', 'study', 'first', 'evidence', 'psychosis', 'risk', 'social', 'performance', 'task', 'activation', 'associative', 'striatum', 'useful', 'behavioral', 'measure', 'psychosis', 'risk']


# Reduce words to numbers

In [ ]:
id2word = corpora.Dictionary(data_words)

corpus = []

for text in tqdm(data_words):
    new = id2word.doc2bow(text)
    corpus.append(new)
    
print(corpus[0])

100%|██████████| 5557/5557 [00:00<00:00, 18481.49it/s]

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 2), (11, 1), (12, 1), (13, 2), (14, 2), (15, 1), (16, 2), (17, 1), (18, 3), (19, 3), (20, 1), (21, 1), (22, 6), (23, 3), (24, 6), (25, 1), (26, 3), (27, 2), (28, 1), (29, 2), (30, 3), (31, 1)]


In [ ]:
for i in range(10):
    print(i, id2word[i])

0 activation
1 addition
2 anhedonia
3 associative
4 behavioral
5 control
6 current
7 disorder
8 elevated
9 evidence


# First simple Model

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=30, #just an initial guess
                                           random_state=100,
                                           update_every=1, #update topic model after each duration
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


# Lower the perplexity better the model.
# Higher the topic coherence, the topic is more human interpretable.


Perplexity:  -13.360019566719181

Coherence Score:  0.3734046178530506


# Visualize and Analyze the Model

In [ ]:
pyLDAvis.enable_notebook()
lda_viz = gensimvis.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
lda_viz

/home/ckoss/miniconda3/envs/abd_workshop_2021/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.118997  0.379320       1        1  17.313304
8      0.308591 -0.023342       2        1  15.003648
15    -0.123412  0.250915       3        1  10.313126
22    -0.082102  0.509667       4        1   7.377309
23     0.458447 -0.204262       5        1   4.973441
21     0.477643 -0.058429       6        1   4.764475
6      0.122892  0.475981       7        1   4.092617
28    -0.413830 -0.300264       8        1   3.399241
27    -0.457224  0.225399       9        1   2.781955
19     0.282100  0.374458      10        1   2.628400
25     0.420379  0.252728      11        1   2.184135
13    -0.097127  0.366339      12        1   2.101249
2     -0.444580  0.057285      13        1   1.885763
16    -0.301489  0.380426      14        1   1.654463
5     -0.024353 -0.274100      15        1   1.639117
18    -0.313490  0.212732      16        1   1.600545
4     -0.089958 -0.019210      17        1   1.562682
10    -0.472177 -0.077227      18        1   1.517480
11     0.109679  0.070491      19        1   1.459228
14     0.453933  0.085173      20        1   1.331701
3     -0.269555 -0.067014      21        1   1.305974
0      0.126380 -0.478907      22        1   1.267476
9      0.267203  0.182453      23        1   1.143582
24    -0.163870 -0.450462      24        1   1.107731
26    -0.326851 -0.212739      25        1   1.067305
12     0.145096 -0.198899      26        1   0.983008
1      0.320452 -0.309218      27        1   0.954991
29    -0.222227 -0.336529      28        1   0.910259
17     0.212864 -0.377574      29        1   0.859019
20    -0.022407 -0.435191      30        1   0.816777, topic_info=           Term         Freq        Total Category  logprob  loglift
158       model  4402.000000  4402.000000  Default  30.0000  30.0000
207        self  3297.000000  3297.000000  Default  29.0000  29.0000
26       social  3448.000000  3448.000000  Default  28.0000  28.0000
264        work  2719.000000  2719.000000  Default  27.0000  27.0000
95       people  2353.000000  2353.000000  Default  26.0000  26.0000
..          ...          ...          ...      ...      ...      ...
4    behavioral     0.038232   932.286656  Topic30 -11.4446  -5.2942
5       control     0.038232  1597.694121  Topic30 -11.4446  -5.8328
6       current     0.038232  1196.147559  Topic30 -11.4446  -5.5434
7      disorder     0.038232  1047.184851  Topic30 -11.4446  -5.4104
8      elevated     0.038232    89.123640  Topic30 -11.4446  -2.9466

[1198 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
123       2  0.043204      ability
123       6  0.765725      ability
123      19  0.190282      ability
622      17  0.988650  abnormality
266       5  0.997593     academic
...     ...       ...          ...
265       2  0.594695         year
265       8  0.092171         year
265      19  0.311868         year
1250     28  0.997747        young
300       3  0.997477        youth

[1217 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 9, 16, 23, 24, 22, 7, 29, 28, 20, 26, 14, 3, 17, 6, 19, 5, 11, 12, 15, 4, 1, 10, 25, 27, 13, 2, 30, 18, 21])

In [ ]:
# are we seeing few overlap of circles (topics)? why?
# are we seeing good representing top words?

# Topic 4: "more", "when", "less" -> not good representer of a topic

# 2nd try - Make the Model better

# Boost word Representation

In [ ]:
#BIGRAMS and TRIGRAMS
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=50)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=50)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return(bigram[doc] for doc in texts)

def make_trigrams(texts):
    return (trigram[bigram[doc]] for doc in texts)

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = [text for text in make_trigrams(data_bigrams)]

In [ ]:
print(data_bigrams_trigrams[0])

['psychosis', 'extent', 'striatal', 'functioning', 'current', 'study', 'psychosis', 'risk', 'social', 'performance', 'task', 'striatal', 'functioning', 'psychosis', 'risk', 'participant', 'elevated', 'score', 'psychosis', 'risk', 'participant', 'interview', 'psychotic', 'like', 'experience', 'risk', 'psychotic_disorder', 'elevated', 'social', 'anhedonia', 'control', 'group', 'group', 'poor', 'performance', 'task', 'addition', 'permag', 'participant', 'psychotic', 'like', 'experience', 'study', 'first', 'evidence', 'psychosis', 'risk', 'social', 'performance', 'task', 'activation', 'associative', 'striatum', 'useful', 'behavioral', 'measure', 'psychosis', 'risk']


In [ ]:
print(data_bigrams_trigrams[1])

['article', 'new', 'approach', 'replication', 'result', 'effect', 'size', 'estimation', 'hypothesis', 'testing', 'extent', 'replication', 'result', 'consistent', 'effect', 'size', 'big', 'detectable', 'original', 'study', 'approach', 'replication', 'finding', 'benefit', 'unsuccessful', 'replication_attempt', 'study', 'noisy', 'effect', 'different', 'true', 'finding', 'underpowered', 'replication', 'compelling', 'inference', 'general', 'revisited', 'replication', 'particular']


In [ ]:
print(data_bigrams_trigrams[2])

['major_depressive', 'disorder', 'devastating', 'disease', 'people', 'productivity', 'health_care', 'wealth', 'research', 'understanding', 'therapy', 'effective', 'symptom', 'disease', 'recent', 'clinical', 'preclinical', 'study', 'genetic', 'screening', 'transgenic', 'rodent', 'major', 'role', 'gene', 'central', 'expression', 'receptor', 'protein', 'individual', 'risk', 'gene', 'brain', 'tissue', 'organism', 'immediate', 'long_term', 'response', 'social', 'environmental', 'stressor', 'primary', 'contributor', 'review', 'current', 'state', 'knowledge', 'physiology', 'occurrence', 'symptom', 'review', 'finding', 'multiple', 'laboratory', 'part', 'symposium', 'topic', 'annual', 'meeting', 'idea', 'datum', 'review', 'great', 'progress', 'past', 'few', 'decade', 'understanding', 'pathway', 'novel', 'treatment', 'detection', 'method', 'disorder', 'right']


In [ ]:
print(data_bigrams_trigrams[3])

['posttraumatic_stress_disorder', 'ptsd', 'regional', 'alteration', 'brain', 'structure', 'function', 'symptom', 'cognitive', 'deficit', 'disorder', 'first', 'systematic', 'meta_analysis', 'neurocognitive', 'outcome', 'broad_range', 'cognitive', 'domain', 'profile', 'cognitive', 'deficit', 'clinical', 'factor', 'study', 'characteristic', 'report', 'datum', 'study', 'participant', 'trauma', 'comparison', 'participant', 'healthy', 'comparison', 'participant', 'trauma', 'exposure', 'effect', 'size', 'estimate', 'effect', 'meta_analysis', 'cognitive', 'domain', 'attention', 'memory', 'executive_function', 'verbal', 'learning', 'verbal', 'memory', 'visual', 'learning', 'visual', 'memory', 'language', 'speed', 'information', 'processing', 'visuospatial', 'ability', 'significant', 'neurocognitive', 'effect', 'magnitude', 'large', 'effect', 'size', 'verbal', 'learning', 'speed', 'information', 'processing', 'attention', 'memory', 'verbal', 'memory', 'effect', 'size', 'estimate', 'large', 'trea

In [ ]:
#TF-IDF Removal  #drawback:may delete important, but frequent words
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)
texts = data_bigrams_trigrams
corpus = [id2word.doc2bow(text) for text in texts]

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03 #good treshold? frequency of word in corpus
words = []
words_missing_in_tfidf = []

for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = []
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids]
    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

# 2nd Model with n_grams and tfidf

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=30, #just an initial guess
                                           random_state=100,
                                           update_every=1, #update topic model after each duration
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


# Lower the perplexity better the model.
# Higher the topic coherence, the topic is more human interpretable.


Perplexity:  -14.78157699443581

Coherence Score:  0.3709570962275056


In [ ]:
pyLDAvis.enable_notebook()
lda_viz = gensimvis.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
lda_viz

/home/ckoss/miniconda3/envs/abd_workshop_2021/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.464937 -0.228822       1        1  15.136905
22     0.116627 -0.378008       2        1  12.493467
17    -0.504507 -0.027528       3        1   6.646888
15     0.382911 -0.265492       4        1   6.062600
1      0.468224 -0.173582       5        1   5.759802
9      0.500558 -0.006399       6        1   5.023230
29     0.466753  0.222923       7        1   4.904451
21     0.161714 -0.489730       8        1   3.970249
28    -0.331287 -0.336256       9        1   3.347053
19    -0.216116 -0.432176      10        1   3.180871
0     -0.290239  0.010500      11        1   2.686784
26    -0.454428  0.161969      12        1   2.345771
20    -0.076101 -0.077824      13        1   2.225405
11    -0.095574 -0.312550      14        1   2.180234
3      0.055220  0.498877      15        1   2.078723
2     -0.157740  0.280447      16        1   2.031667
13    -0.361602  0.333027      17        1   1.960304
25    -0.300652 -0.144150      18        1   1.842949
12    -0.039992 -0.463464      19        1   1.739074
14     0.269452 -0.327036      20        1   1.656636
5      0.141999 -0.130912      21        1   1.622495
4      0.183782  0.412488      22        1   1.548069
8     -0.290258  0.171404      23        1   1.533051
10     0.038892  0.212985      24        1   1.325090
23    -0.205364  0.434446      25        1   1.292780
18    -0.044243  0.407295      26        1   1.206310
27     0.164790  0.042929      27        1   1.182754
24     0.346512  0.053679      28        1   1.142002
7      0.240732  0.212245      29        1   0.950685
16     0.294877  0.338718      30        1   0.923700, topic_info=            Term         Freq        Total Category  logprob  loglift
40        effect  4143.000000  4143.000000  Default  30.0000  30.0000
26        social  3234.000000  3234.000000  Default  29.0000  29.0000
155        model  4063.000000  4063.000000  Default  28.0000  28.0000
177     behavior  2580.000000  2580.000000  Default  27.0000  27.0000
13         group  2547.000000  2547.000000  Default  26.0000  26.0000
..           ...          ...          ...      ...      ...      ...
10        extent     0.037091   385.018511  Topic30 -11.4935  -4.5631
11         first     0.037091   734.685159  Topic30 -11.4935  -5.2093
12   functioning     0.037091   151.117139  Topic30 -11.4935  -3.6279
13         group     0.037091  2547.765535  Topic30 -11.4935  -6.4528
14     interview     0.037091   147.309423  Topic30 -11.4935  -3.6024

[1117 rows x 6 columns], token_table=      Topic      Freq                  Term
term                                       
122       8  0.998590               ability
1259      5  0.715630                  able
1259     17  0.279671                  able
399      24  0.982454  academic_achievement
1045     11  0.958430         accessibility
...     ...       ...                   ...
259       2  0.565196                  year
259       5  0.338357                  year
259      24  0.081104                  year
1302     22  0.995192                 young
295       4  0.996854                 youth

[1070 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 23, 18, 16, 2, 10, 30, 22, 29, 20, 1, 27, 21, 12, 4, 3, 14, 26, 13, 15, 6, 5, 9, 11, 24, 19, 28, 25, 8, 17])

In [ ]:
# less overlap, but a lot of non-representative words